In [1]:
%pip install groq # Instalando o groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.3 MB/s eta 0:00:00


In [2]:
import os

os.environ['GROQ_API_KEY'] = "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" # Utilizando a chave da API que peguei no Groq

In [3]:
import os
from groq import Groq # Importa a classe que nos permite interagir com a API do Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
) # Inicializando o Groq e passando a minha chave ali de cima

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models", # Passando as mensagens para o nosso modelo
        }
    ],
    model="llama3-70b-8192", # Definindo que modelo vamos utilizar
)

print(chat_completion.choices[0].message.content) # Aqui printamos a resposta que o modelo vai dar para o nosso prompt/mensagem que passamos para ele ali em cima

Fast language models are crucial in today's natural language processing (NLP) landscape, and their importance can be seen in various aspects:

1. **Real-time Applications**: Fast language models enable real-time applications, such as chatbots, virtual assistants, and language translation systems, to respond quickly and efficiently to user inputs.
2. **Improved User Experience**: Fast response times in language models lead to a more engaging and interactive user experience, increasing user satisfaction and loyalty.
3. **Scalability**: Fast language models can handle large volumes of text data, making them scalable for applications that require processing massive amounts of text, such as social media monitoring or customer feedback analysis.
4. **Efficient Inference**: Fast language models reduce the computational resources required for inference, making them more energy-efficient and cost-effective, especially in cloud-based or edge computing environments.
5. **Low-Latency Edge AI**: Fa

In [4]:
class Agent:
  def __init__(self, client, system):
    self.client =  client # Inicializando o cliente
    self.system = system # Inicializando a mensagem do sistema para o modelo entender como ele deve trabalhar em loop
    self.messages = [] # Lista de mensagens que vamos passar
    if self.system is not None: # Se a mensagem não for vazia nos vamos adiciona-la na nossa lista
      self.messages.append({"role":"system", "content": self.system})

  def __call__(self, message=""):
      if message: # Vai ser ativado sempre que o nosso agente for chamado
        self.messages.append({"role":"user", "content": message}) # Se receber uma mensagem vai adiciona-la na lista
      result = self.execute() # Depois de receber a mensagem vamos executar o agente com todas as mensagens da nossa lista
      self.messages.append({"role":"assistant", "content": result}) # Depois vai adicionar o resultado na nossa lista
      return result # E por fim retornar o resultado para gente

  def execute(self): # Método de execução que usamos no método anterior após recebermos a mensagem
      completion = client.chat.completions.create(
          messages= self.messages ,
          model="llama3-70b-8192", # Definindo que modelo vamos utilizar
      )
      return completion.choices[0].message.content

In [5]:
system_prompt  = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.9736e+24

Thought: I need to multiply this by 2
Action: calculate: 5.9736e+24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944e+25

If you have the answer, output it as the Answer.

Answer: The mass of the Earth times 2 is 1,1944x10e25

Now it's your turn:
""".strip()

# Esse o prompt do sistema que vai fazer o agente entender como ele deve se comportar para seguir o loop

In [6]:
# Ferramentas
def calculate(operation):
  return eval(operation) # Ferramenta padrão de calculadora

def get_planet_mass(planet) -> float:
 match planet.lower():
    case "earth":
      return 5.9736e+24 # Ferramenta para termos as massas dos planetas sem termos que pesquisar na internet, tipo um banco de dados
    case "jupiter":
      return 1.898e27
    case "mars":
      return 6.39e23
    case "mercury":
      return 3.285e23
    case "neptune":
      return 1.024e26
    case "saturn":
      return 5.685e26
    case "uranus":
      return 8.683e25
    case "venus":
      return 4.868e24

In [7]:
neil_tyson = Agent(client, system_prompt) # Instância do nosso agente

**RODANDO O AGENTE MANUALMENTE NAS PRÓXIMAS CÉLULAS**


In [8]:
result = neil_tyson("What is the mass of Earth times 5?") # Dando um prompt teste para o agente
print(result) # Printando o resultado para vermos se está tudo de acordo

Thought: I need to find the mass of Earth.


In [9]:
neil_tyson.messages # Aqui é um log das mensagens que o agente recebe e os pensamentos que ele cria

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.9736e+24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.9736e+24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944e+25\n\nIf you have the

In [11]:
result = neil_tyson() # Aqui ele vai definir que ação ele deve tomar
print(result) # Printamos para conferir se está tudo certo

Action: get_planet_mass: Earth
PAUSE


In [12]:
observation = get_planet_mass("Earth") # Como ainda não criamos o loop já damos a resposta da ação dele diretamente para ele
print(observation) # Printamos para conferir se está tudo certo

5.9736e+24


In [13]:
next_prompt = f"Observation:{observation}"
result = neil_tyson(next_prompt) # Dando a observação para o agente
print(result) # Printamos para conferir se está tudo certo

Thought: I need to multiply this by 5


In [14]:
neil_tyson.messages # Conferindo o log para termos certeza que o loop está de acordo, Thought, Action e Observation.

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.9736e+24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.9736e+24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944e+25\n\nIf you have the

In [16]:
result = neil_tyson() # Esperamos a próxima ação
print(result) # Printamos para conferir se está tudo certo

Action: calculate: 5.9736e+24 * 5
PAUSE


In [17]:
observation = calculate("5.9736e24 * 5") # Criando a observação para ele
print(observation) # Printamos para conferir se está tudo certo

2.9868e+25


In [18]:
next_prompt = f"Observation:{observation}"
result = neil_tyson(next_prompt) # Colocando a observação nele
print(result) # Printamos para conferir se está tudo certo

Answer: The mass of the Earth times 5 is 2.9868e+25


In [19]:
neil_tyson.messages # Conferindo o log pela última vez

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.9736e+24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.9736e+24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944e+25\n\nIf you have the

In [20]:
import re # Criando o loop do agente

def agent_loop(max_iterations, system, query):
  agent = Agent(client, system_prompt) # Inicilizando o agente
  tools = ['calculate', 'get_planet_mass'] # Inicilizando as ferramentas que temos
  next_prompt = query # Definimos o primeiro prompt como pedido que vamos mandar para o loop do agente
  i = 0
  while i < max_iterations: # Iniciando o loop
    i += 1
    result = agent(next_prompt) # Chamando o agente
    print(result) # Printando o resultado para análise

    if "PAUSE" in result and "Action" in result: # Se o resultado tiver PAUSE e AÇÃO
      action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE) # Nós encontramos o nome da ação e os parâmetros com essa linha de código
      chosen_tool = action[0][0] # Organizando a ferramenta selecionada
      arg = action[0][1] # Organizado o parâmetro encontrado

      if chosen_tool in tools: # Se a ferramenta estiver entre as que nós temos
        result_tool = eval(f"{chosen_tool}('{arg}')") # Usamos e ferramenta e temos o resultado dela
        next_prompt = f"Observation: {result_tool}" # Que é analisado nessa observação

      else:
        next_prompt = "Observation: Tool not found" # Se não tivermos a ferramenta vamos dizer que não encontramos a ferramenta para que o sistema procure outra correta

      print(next_prompt) # Printamos para sabermos o que está acontecendo
      continue # Continua a fazer mais iterações

    if "Answer" in result: # Se o agente tiver "resposta" no seu resultado nós paramos o loop
      break


In [23]:
agent_loop(max_iterations=10, system=system_prompt, query="What is the mass of the Earth plus the mass of Mercury and all of it times 5?") # Implementando o loop do agente criado

Thought: I need to find the mass of Earth and Mercury, then add them together and multiply by 5.
Action: get_planet_mass: Earth
PAUSE
Observation: 5.9736e+24
Thought: Now I have the mass of Earth, I need to get the mass of Mercury.
Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: Now I have the masses of both Earth and Mercury, I need to add them together.
Action: calculate: 5.9736e+24 + 3.285e+23
PAUSE
Observation: 6.3021e+24
Thought: Now I have the sum of the masses of Earth and Mercury, I need to multiply it by 5.
Action: calculate: 6.3021e+24 * 5
PAUSE
Observation: 3.15105e+25
Answer: The mass of the Earth plus the mass of Mercury and all of it times 5 is 3.15105e+25.
